# Text classification using FastText

First of all we should prepare our text data in format where each block starts with "\__label__"

In [30]:
import re

label = re.compile(r'LABEL=\d+')
operator = re.compile(r'OPER \[.+\] .+$')
with open("test.txt") as inp:
    with open("test_prep.txt", 'w') as outp:
        for dialog in inp.read().split('\n\n'):
            for line in dialog.split('\n'):
                pos = line.find("'|'")
                if pos > -1:
                    line = line[:pos]
                for lab in label.findall(line):                    
                    outp.write("__label__%s" %lab[6])
#                     print("__%s__" %lab.lower())
                for oper in operator.findall(line):
                    outp.write(' ')
                    outp.write(oper[20:])
#                     print(oper[20:])
            outp.write('\n')

In [35]:
label = re.compile(r'LABEL=\d+')
operator = re.compile(r'OPER \[.+\] .+$')
with open("train.txt") as inp:
    with open("train_prep.txt", 'w') as outp:
        for dialog in inp.read().split('\n\n'):
            for line in dialog.split('\n'):
                pos = line.find("'|'")
                if pos > -1:
                    line = line[:pos]
                for lab in label.findall(line):                    
                    outp.write("__label__%s" %lab[6])
#                     print("__%s__" %lab.lower())
                for oper in operator.findall(line):
                    outp.write(' ')
                    outp.write(oper[20:])
#                     print(oper[20:])
            outp.write('\n')
    

In [3]:
import fasttext

In [43]:
model = fasttext.train_supervised(input="train_prep2.txt", autotuneValidationFile='valid_prep2.txt')

Progress: 100.0% Trials:   13 Best score:  0.925000 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  16756
Number of labels: 2
Progress: 100.0% words/sec/thread:  352077 lr:  0.000000 avg.loss:  0.123118 ETA:   0h 0m 0s


In [44]:
model.test("test_prep.txt")

(484, 0.9318181818181818, 0.9318181818181818)

In [40]:
import numpy as np

In [47]:
y_true = np.array([])
y_pred = np.array([])
with open("test_prep.txt", "r", encoding="utf-8") as f:
    for line in f:
        if len(line) > 11:
            labels, probs = model.predict(line.strip())
            if labels[0] == "__label__0":
                y_pred = np.append(y_pred, 0)
            else:
                y_pred = np.append(y_pred, 1)
            if line.startswith("__label__1"):
                y_true = np.append(y_true, 1)
            else:
                y_true = np.append(y_true, 0)

In [48]:
from sklearn.metrics import f1_score
f1_score(y_true, y_pred)

0.9193154034229829